In [1]:
import pyam 
import pandas as pd
import numpy as np

from pathlib import Path
import os
import dotenv

Step 1: Load the AR6 data and metadata

In [2]:
dotenv.load_dotenv()

True

In [3]:
df = pd.read_csv(
    os.environ['AR6'],
    index_col=[0, 1]
)

In [4]:
meta = pd.read_excel(
    os.environ['AR6_META'],
    sheet_name='meta_Ch3vetted_withclimate',
    index_col=[0,1]
)

Step 2: Filter out for the kyoto greenhouse gas emissions (harmonised). We will use this data to calculate the cumulative net-negative GHG emissions.

In [5]:
df_kyoto = df[
    df['Variable']=='AR6 climate diagnostics|Infilled|Emissions|Kyoto Gases (AR6-GWP100)'
]

Step 3: Filter the metadata using:
* Median peak warming less than 2°C
* Achieve net zero GHG before 2100

In [6]:
meta_two = meta[
    meta['Median peak warming (MAGICCv7.5.3)']<2
]

In [7]:
meta_two_netzero = meta_two[
    meta_two['Year of netzero GHG emissions (Harm-Infilled) Table SPM2']<2100
]

Step 4: Downselect scenarios in the the dataframe.

In [8]:
df_kyoto_downselected = pyam.IamDataFrame(
    df_kyoto.loc[meta_two_netzero.index, :]
    .reset_index()
)

Step 5: Convert units and then calculate the cumulative net-negative GHG emissions.

In [9]:
df_kyoto_downselected.convert_unit(
    current='Mt CO2-equiv/yr',
    to='Gt CO2-equiv/yr',
    inplace=True
)

In [10]:
df_kyoto_downselected.set_meta(
    df_kyoto_downselected.timeseries().apply(
        lambda x: pyam.timeseries.cumulative(
            x,
            first_year=meta_two_netzero.loc[x.name[0:2],'Year of netzero GHG emissions (Harm-Infilled) Table SPM2'],
            last_year=2100
        ),
        axis=1
    ),
    name='cumulative_netneg_gtco2'
)

Step 6: Add in the additional columns to the metadata and then save this out for plotting.

In [11]:
panelc_data = df_kyoto_downselected.meta

In [12]:
panelc_data.loc[:, 'category'] = meta_two_netzero.loc[panelc_data.index, 'Category']
panelc_data.loc[:, 'warming_peak'] = meta_two_netzero.loc[panelc_data.index, 'Median peak warming (MAGICCv7.5.3)'].round(2)
panelc_data.loc[:, 'warming_2100'] = meta_two_netzero.loc[panelc_data.index, 'Median warming in 2100 (MAGICCv7.5.3)'].round(2)

Step 7: Save this out.

In [13]:
panelc_data.to_excel(
    Path('data/03_data.xlsx'),
    merge_cells=False
)